In [19]:
import time
import pandas as pd
import torch
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
torch.cuda.is_available()

1
Tesla K80


True

In [ ]:
!pip install transformers
!pip3 install emoji

In [11]:
from transformers import AutoTokenizer, AutoModel

import torch

inputlength = 128


class BERTTweet:

    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False, normalization=True)
        self.bertweet = AutoModel.from_pretrained("vinai/bertweet-base").to('cuda:0')

    def tweet_to_vec_string(self, tweet):
        vec = self.tweet_to_vec(tweet)
        return torch.flatten(vec).tolist()

    def tweet_to_vec(self, tweet):
        tokens = self.tokenizer.encode(tweet)

        if len(tokens) > inputlength:
            tokens = tokens[0:inputlength]

        input_ids = torch.tensor([tokens]).to('cuda:0')

        with torch.no_grad():
            features = self.bertweet(input_ids)

        return features.pooler_output


In [14]:
import time
import pandas as pd


class BERTCalculator:

    def __init__(self):
        self.tweet_bert = BERTTweet()

    def pre_processing(self, tweet):
        tweet = tweet.replace('\n', '')
        tweet = tweet.replace('\t', '')
        tweet = tweet.replace('\r', '')
        return tweet

    def pipeline(self, index, row, data_set):
        tweet = row["text"]

        tweet = self.pre_processing(tweet)

        vec = self.tweet_bert.tweet_to_vec_string(tweet)
        data_set._set_value(index, 'bert', vec)


def vectorize_tweets(data):
    print("VectorizeBert started")
    mechanism = BERTCalculator()

    data["bert"] = [[]] * len(data)
    t0 = time.time()

    for index, row in data.iterrows():

        try:
            mechanism.pipeline(index, row, data)

        except IndexError as err:
            print(err)

        except Exception as err:
            print(row["text"])
            raise err

        if index % 1000 == 0:
            print("index", index, " time: ", (time.time() - t0))

    print("VectorizeBert done")

    return data


In [ ]:
root_path = "/content/drive/MyDrive/Dev/"
file = "03 2018_en"
try:
    data = pd.read_csv(root_path + file + ".csv", sep=";")
except:
    data = pd.read_csv(root_path + file + ".csv", lineterminator="\n")


print("Loaded", file, " ", data.shape)

data = vectorize_tweets(data)
data.to_csv(path_or_buf=root_path + file + "_bert.csv")


Loaded 03 2018_en   (958245, 13)
VectorizeBert started


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
bert = ""
for i in range(0,768):
    bert += ",bert" +str(i)

bert += ","

with open("../Data/2018tweets/Objects/(60Min).csv", 'r') as infile, \
     open('../Data/2018tweets/Objects/(60Min)bert.csv', 'w') as outfile:
    data = infile.read()
    data = data.replace('"', '')
    data = data.replace('[', '')
    data = data.replace(']', '')
    data = data.replace(',bert,', bert)

    outfile.write(data)
